In [ ]:
!pip install langchain langchain-community #diff type of doc we use so langchain
!pip install faiss-cpu #facebook ai similarity search
!pip install pypdf python-docx
!pip install sentence-transformers #our data is niche, always text so (sentence-) this is specified
!pip install transformers #encoding decoding data

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.7 MB/s eta 0:00:00


In [12]:
from google.colab import files #giving access to take file from system kernal to jupyterkernal
uploaded=files.upload() #when i run it will ask to choose file

file_path=list(uploaded.keys())[0] #0th element is accessed and stored all in list
print("Uploaded",file_path)

Saving readme.txt to readme (1).txt
Uploaded readme (1).txt


In [13]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#pick loader based on file type
if file_path.endswith(".pdf"):
 loader = PyPDFLoader(file_path)
elif file_path.endswith(".docx"):
 loader = Docx2txtLoader(file_path)
else:
  loader =TextLoader(file_path)

docs=loader.load()

#split the doc into smaller chunks so model doesn’t get overloaded
splitter= RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=splitter.split_documents(docs)

print(f"Total chunks:{len(documents)}")

Total chunks:9


In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

#small + fast embeddings
embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorspace= FAISS.from_documents(documents,embeddings)

In [15]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

#load small model
flan_pipeline=pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)
llm=HuggingFacePipeline(pipeline=flan_pipeline)

Device set to use cpu


In [16]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorspace.as_retriever(search_kwargs={"k":3}),
    chain_type="stuff"
)

query = "Give me a short summary of the document"
print(qa.run(query))

Exploratory Analysis Performed queries to summarize key variables, such as the total number of layoffs, industry, and country. 3. Exploratory Analysis Performed queries to summarize key variables, such as the total number of layoffs, industry, and country.


In [17]:
while True:
  q=input("Ask a question (or 'exit'):")
  if q.lower()=="exit":
    print("Thanks for using my QA bot! Bye 👋")
    break
  print("Answer:",qa.run(q))

Ask a question (or 'exit'):give three important points from document
Answer: Exploratory Analysis Performed queries to summarize key variables, such as the total number of layoffs, industry, and country. Identify and handle issues: Detect missing or inconsistent data and handle them appropriately. Prepare data for further analysis, visualization, or modeling. Standardize formats: Ensure that text columns and date columns are standardized. Steps Performed in Data Cleaning 1. Data Inspection Loaded the dataset and examined the first few rows to understand its structure. Date: The date when the layoffs occurred. Stage: The stage or phase of the layoff. Country: The country in which the company operates. Funds Raised (Millions): The amount of funds raised by the company (in millions).
Ask a question (or 'exit'):who are you?
Answer: a human
Ask a question (or 'exit'):are you mad?
Answer: No, I'm not mad.
Ask a question (or 'exit'):exit
Thanks for using my QA bot! Bye 👋
